In [1]:
%pip install tensorflow opencv-python mediapipe scikit-learn matplotlib pandas pyarrow wandb plotly

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/d9/64/7fdfb9386511cd6805451e012c537073a79a958a58795c4e602e538c388c/opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/c9/e4/3e645a8f87577553194a2a15383a60d61b8381cf864e903e43e4c6eb58e0/mediapipe-0.10.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/4e/53/14405a47292b59235d811a2af8634aba188ccfd1a38ef4b8042f3447d79a/scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaini

In [3]:
import os
import math
import pandas as pd
import numpy as np
#Load parquet data into dataset_parquet for training.
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from glob import glob
from tqdm import tqdm
from collections import Counter
import random
import time
import wandb

In [4]:
class CONFIG:
    # root = os.path.join("/", "kaggle", "input", "asl-signs") 
    #root = os.path.join("..","data", "ASL-ds")
    root = os.path.join(".")
    DATA_LIMIT = 2000
    BATCH_SIZE = 16
    VIDEO_LENGTH = 36
    TRAIN_VAL_SPLIT = 0.80
    WANDB_RUN = "mediapipe-asl-dataset"


In [5]:
LIPS_IDXS0 = np.array([
        61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
        291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
        78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
        95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    ])

EYE_LEFT = np.array([33, 7, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 471, 470, 469, 472])
EYE_RIGHT = np.array([362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 476, 475, 474, 477])
# Landmark indices in original data
LEFT_HAND_IDXS0 = np.arange(468,489)
RIGHT_HAND_IDXS0 = np.arange(522,543)
LEFT_POSE_IDXS0 = np.array([502, 504, 506, 508, 510])
RIGHT_POSE_IDXS0 = np.array([503, 505, 507, 509, 511])

print("LIPS : ",len(LIPS_IDXS0))
print("EYE_LEFT : ",len(EYE_LEFT))
print("EYE_RIGHT : ",len(EYE_RIGHT))
print("LEFT_HAND : ",len(LEFT_HAND_IDXS0))
print("RIGHT_HAND : ",len(RIGHT_HAND_IDXS0))
print("LEFT_POSE : ",len(LEFT_POSE_IDXS0))
print("RIGHT_POSE : ",len(RIGHT_POSE_IDXS0))

all_selection = np.concatenate([LIPS_IDXS0, EYE_LEFT, EYE_RIGHT, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, LEFT_POSE_IDXS0, RIGHT_POSE_IDXS0])
print(len(all_selection))

LIPS :  40
EYE_LEFT :  20
EYE_RIGHT :  20
LEFT_HAND :  21
RIGHT_HAND :  21
LEFT_POSE :  5
RIGHT_POSE :  5
132


In [6]:
#this code sorts out a parquet files and rearrange the order to pose,face, left-hand, right-hand
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objs as go
import plotly.io as pio

ids = None

order_global = {"pose" : 10000, "face" : 1000, "left_hand" : 100, "right_hand" : 10}

def visualize_keypoints(frames : np.ndarray, point_size : int):
    if len(frames.shape) == 1:
        frames = np.array([frames])
    
    for frame in frames:
        frame = frame.reshape(-1, 3)
        sizes = point_size * np.ones(frame.shape[0])

        fig = go.Figure(data=go.Scatter(x=frame[:,0], y=2.5 - frame[:,1], mode='markers',
                                        marker=dict(
                                            size=sizes
                                            )))

    # Customize the layout
    fig.update_layout(title='visualization of human keypoints',
                        xaxis_title='',
                        yaxis_title='',
                        width=1000,
                        height=1600)

    fig.update_xaxes(range=[-0.2, 1.4])  # Set x-axis range from 0 to 6
    fig.update_yaxes(range=[0, 2.5])  # Set y-axis range from 10 to 20

    # Show the plot
    fig.show()


def process_parquet(ds, idxes = None):
    ret = []
    frames_unique = sorted(np.unique(ds["frame"]))
    for i,frame in enumerate(frames_unique):
        frame_ds = ds[ds['frame'] == frame]
        
        order = []
        for el in frame_ds["row_id"]:
            _frame, part, keypoint = el.split("-")
            order.append(order_global[part] - int(keypoint))

        order = np.array(order)
        frame_ds.iloc[:, 1] = order
        frame_ds = frame_ds.sort_values(by="row_id", ascending=False)
    
        vals = np.array(frame_ds[["x", "y", "z"]])
        if idxes is not None:
            vals = vals[idxes]
    
        vals = vals.flatten()

        ret.append(vals)
        
    return np.array(ret)


def process_parquet2(ds, idxes = None):
    ret = []    
    frame_size = 543
    it = len(ds) // frame_size
    assert it == len(ds) / frame_size
    
    for i in range(it):
        vals = ds.iloc[ i * frame_size : (i + 1 ) * frame_size ]        
        
        if idxes is not None:          
            vals = ds.iloc[idxes]
                        
        ret.append(np.array(vals[["x","y", "z"]]).flatten())
        
    return np.array(np.array(ret))

In [7]:
from sklearn.model_selection import train_test_split

#custom class to load data from Parquet files for training ML models.
class ParquetDatasetCached(keras.utils.Sequence):
    def __init__(self, dataset_folder, csv_file : str, batch_size=CONFIG.BATCH_SIZE, 
                 data_limit :int= CONFIG.DATA_LIMIT, check_if_file_exists = True, 
                 preprocessing_func=None, frame_length :int = CONFIG.VIDEO_LENGTH,
                 split : str = "train", train_val_split : float = CONFIG.TRAIN_VAL_SPLIT,
                 sort_by_counts : bool = True, **kwargs
                ):
        super().__init__(**kwargs)
        #taking keras sequence for .fit(), .evaluate(), .predict() methods
        #load csv - it has the path to parquet file, and another to store label
        self.csv_path = csv_file
        self.root_folder = dataset_folder
        self.batch_size = batch_size
        #optional pre-processing function to the parquet files.
        self.preprocessing_func = preprocessing_func
        
        self.csv_data = pd.read_csv(self.csv_path)
        
        self.all_files = []
        self.not_exists = []
        self.frame_length = frame_length

        
        for path, label in tqdm(list(zip(self.csv_data["path"], self.csv_data["sign"]))):
            prop_path = os.path.join(self.root_folder, path)
            
            if check_if_file_exists:
                if os.path.exists(prop_path):
                    self.all_files.append((prop_path, label))
                else:
                    self.not_exists.append(prop_path)
            else:
                self.all_files.append((prop_path, label))
                
                    
        self.all_files = np.array(self.all_files)
        self.unique_labels = np.unique(self.all_files[:, 1])
        self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
    
        # sort the values by popularity
        if sort_by_counts:
            cnt = Counter(self.all_files[:, 1])
            vals = []
            
            for i,row in enumerate(self.all_files):
                vals.append((int(1e6 * cnt[row[1]] + self.label_2_id [row[1]]),i))
            
            vals = np.array(sorted(vals)[::-1])
            self.all_files = self.all_files[vals[:,1]]

        
        if data_limit < 0:
            train_ds, val_ds = train_test_split(self.all_files, train_size=train_val_split, random_state=42)
        else:
            train_ds, val_ds = train_test_split(self.all_files[:data_limit], train_size=train_val_split, random_state=42)
            self.unique_labels = np.unique(self.all_files[:data_limit, 1])
            self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
            
        if split.lower() == "train":
            self.dataset = train_ds
            
        elif split.lower() == "val":
            self.dataset = val_ds 
            
        else:
            raise Exception("please specify split to be either train or val")
            
        self.cache_data()
                   

    def cache_data(self):
        self.cached_X, self.cached_Y = [], []
        
        pb = tqdm(range(len(self.dataset)), desc="Cacheing")

        for i in pb:
            
            path, label = self.dataset[i]
            df = pd.read_parquet(path)
                        
            one_hot_encoded_label = np.zeros(len(self.unique_labels))
            one_hot_encoded_label[self.label_2_id[label]] = 1

            self.cached_X.append(process_parquet2(df)) 
            self.cached_Y.append(one_hot_encoded_label)        

    def __len__(self):
        # Assuming each Parquet file should be one batch; adjust if necessary
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.preprocessing_func(self.cached_X[idx]), self.cached_Y[idx]                
        
    def on_epoch_end(self):
        # Shuffle files for the next epoch
        np.random.shuffle(self.dataset)


def preprocess(frames):
    current_length, num_features = frames.shape

    if current_length >= CONFIG.VIDEO_LENGTH:
            # TODO: a better than uniform value ? Could place gaussian in the middle
            random_start = random.randint(0, current_length - CONFIG.VIDEO_LENGTH)
            return np.nan_to_num(frames[random_start : (random_start + CONFIG.VIDEO_LENGTH)])
        
    # padd the video to contain zeros 
    return np.concatenate([np.nan_to_num(frames), np.zeros((CONFIG.VIDEO_LENGTH - current_length, num_features))], axis=0)

    
# Usage example
parquet_folder_path = CONFIG.root
train_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                check_if_file_exists = True,
                                split="train")

val_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                 check_if_file_exists= True,
                                 split="val")

print(f"cardinality of train : {len(train_dataset_parquet)}, cardinality of validation : {len(val_dataset_parquet)}")

Cacheing: 100%|██████████| 400/400 [00:05<00:00, 78.58it/s]

cardinality of train : 1600, cardinality of validation : 400


In [8]:
X_shape = train_dataset_parquet[0][0].shape
Y_shape = train_dataset_parquet[0][1].shape
print(f"X_shape = {X_shape}, Y_shape = {Y_shape}")

X_shape = (36, 1629), Y_shape = (5,)


In [9]:
print(train_dataset_parquet.label_2_id)

{'donkey': 0, 'listen': 1, 'look': 2, 'mouse': 3, 'shhh': 4}


In [10]:
def dataset_train_generator():
    # Instantiate your existing dataset loader

    for i in range(len(train_dataset_parquet)):
        X_batch, Y_batch = train_dataset_parquet[i]
        yield X_batch, Y_batch
        
def dataset_val_generator():
    # Instantiate your existing dataset loader

    for i in range(len(val_dataset_parquet)):
        X_batch, Y_batch = val_dataset_parquet[i]
        yield X_batch, Y_batch        


train_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_train_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_val_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

2024-04-24 21:08:03.169740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1002 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [11]:
start = time.time()
isnans =False

f = True
labels_batches = []
for el in tqdm(train_dataset):
    if f:
        print(el[0].shape, el[1].shape)
        f = False
    labels_batches.append(el[1])
        
    isnans |= np.any(np.isnan(el[0]))
    if isnans:
        print("FOUND NAN!")
        break


print(f"Iterating through dataset took : {round( time.time() - start , 4)}s")

25it [00:00, 130.84it/s]

(16, 36, 1629) (16, 5)


100it [00:00, 193.75it/s]

Iterating through dataset took : 0.5199s


# Train experiment

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from wandb.keras import WandbMetricsLogger
from keras.callbacks import Callback
import tensorflow as tf
import wandb


class CosineAnnealingLearningRateScheduler(Callback):
    def __init__(self, max_lr, min_lr, T_max):
        super(CosineAnnealingLearningRateScheduler, self).__init__()
        self.max_lr = max_lr  # Maximum learning rate (i.e., start learning rate)
        self.min_lr = min_lr  # Minimum learning rate
        self.T_max = T_max    # Specifies the number of epochs per cycle
        self.t = 0            # Current epoch

    def on_epoch_begin(self, epoch, logs=None):
        self.t += 1
        cos = np.cos(np.pi * (self.t % self.T_max) / self.T_max)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + cos)

        keras.backend.set_value(self.model.optimizer.lr, lr)

def keras_train(model, filepath : str, max_lr = 1e-4, min_lr = 2.5e-5, T_max=101, epochs=200, run_name="",
                mediapipe_features = "all", USE_WANDB=True,
               verbose="auto"): 
    
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                                 monitor="val_categorical_accuracy",
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode="max",
                                                 save_freq="epoch")
    
    cosine_annealer = CosineAnnealingLearningRateScheduler(max_lr=max_lr,
                                                           min_lr=min_lr,
                                                           T_max=T_max)
    
    #Adam Optimizer - fixed learning rate.
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=max_lr, clipnorm=1.)

    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    
    callbacks  = [checkpoint, cosine_annealer]
    
    if USE_WANDB:
        wandb.init(project=CONFIG.WANDB_RUN,
                        name=run_name,
                        notes="Model summary : \n",
                        config={"max_lr" : max_lr, 
                                "min_lr" : 5e-5, 
                                "scheduler" : "cosineAnnealer", 
                                "epochs" : epochs, 
                                "T_max" : T_max, 
                                "train_size" : len(train_dataset_parquet.dataset),
                                "val_size" : len(val_dataset_parquet.dataset),
                                "unique_classes" : len(train_dataset_parquet.unique_labels), 
                                "video_length" : CONFIG.VIDEO_LENGTH,
                                "features" : mediapipe_features
                                })
        callbacks.append(WandbMetricsLogger())


    
    history = model.fit(train_dataset, epochs=epochs, validation_data = val_dataset, callbacks=callbacks, verbose=verbose)
    
    if USE_WANDB:      
        wandb.finish()
    
    return history

## LSTM

In [27]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM16-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM16-LR-Dense256-fs=36-cls=",
            USE_WANDB=True)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 16)                105344    
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 16)                0         
                                                                 
 dense_26 (Dense)            (None, 256)               4352      
                                                                 
 dense_27 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 110981 (433.52 KB)
Trainable params: 110981 (433.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/200
    100/Unknown - 2s 5ms/step - loss: 1.6213 - categorical_accuracy: 0.2100

2024-04-24 18:25:34.689352: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11847579756920092354
2024-04-24 18:25:35.213983: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11256574607164290797
2024-04-24 18:25:35.214007: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10123891644433725201
2024-04-24 18:25:35.214012: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 566830789400684550
2024-04-24 18:25:35.214019: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13197248710330269184


INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=36-cls=5.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=36-cls=5.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 51ms/step - loss: 0.6215 - categorical_accuracy: 0.7600 - val_loss: 0.6681 - val_categorical_accuracy: 0.7600
Epoch 170/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5739 - categorical_accuracy: 0.8006 - val_loss: 0.6553 - val_categorical_accuracy: 0.7400
Epoch 171/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5845 - categorical_accuracy: 0.7806 - val_loss: 0.6874 - val_categorical_accuracy: 0.7525
Epoch 172/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5756 - categorical_accuracy: 0.7925 - val_loss: 0.6251 - val_categorical_accuracy: 0.7525
Epoch 173/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5988 - categorical_accuracy: 0.7800 - val_loss: 0.6654 - val_categorical_accuracy: 0.7575
Epoch 174/200
100/100 [==============================] - 1s 6ms/step - loss: 0.6041 - categorical_accuracy: 0.7700 - val_loss: 0.6874 - val_categorical_accuracy: 0.7325

INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 54ms/step - loss: 0.5789 - categorical_accuracy: 0.7850 - val_loss: 0.6642 - val_categorical_accuracy: 0.7725
Epoch 180/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5721 - categorical_accuracy: 0.7894 - val_loss: 0.6517 - val_categorical_accuracy: 0.7500
Epoch 181/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5705 - categorical_accuracy: 0.7850 - val_loss: 0.6555 - val_categorical_accuracy: 0.7400
Epoch 182/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5682 - categorical_accuracy: 0.7869 - val_loss: 0.6752 - val_categorical_accuracy: 0.7375
Epoch 183/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5591 - categorical_accuracy: 0.7956 - val_loss: 0.6364 - val_categorical_accuracy: 0.7550
Epoch 184/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5714 - categorical_accuracy: 0.7875 - val_loss: 0.6652 - val_categorical_accuracy: 0.7475

epoch/categorical_accuracy,▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▆▆▇▇▇▇▇▇▇███████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,██▆▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▁▃▃▃▄▅▅▅▆▆▆▆▆▇▆▇▇▇▇▆▇▇▇▇▇▇█▇▇▇▇████████
epoch/val_loss,██▆▅▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁
epoch/categorical_accuracy,0.79438
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.54455
epoch/val_categorical_accuracy,0.7475


In [28]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM32-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM32-LR-Dense256-fs=36-cls=5",
            USE_WANDB=True)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 32)                212736    
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 32)                0         
                                                                 
 dense_28 (Dense)            (None, 256)               8448      
                                                                 
 dense_29 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 222469 (869.02 KB)
Trainable params: 222469 (869.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/200
     91/Unknown - 2s 5ms/step - loss: 1.6322 - categorical_accuracy: 0.2019

2024-04-24 18:31:07.546871: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3296895248915142635
2024-04-24 18:31:07.546894: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 776399685450183940
2024-04-24 18:31:07.546905: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11847579756920092354


INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Dense256-fs=36-cls=5.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Dense256-fs=36-cls=5.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 51ms/step - loss: 0.4182 - categorical_accuracy: 0.8512 - val_loss: 0.5498 - val_categorical_accuracy: 0.7950
Epoch 195/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4148 - categorical_accuracy: 0.8550 - val_loss: 0.6932 - val_categorical_accuracy: 0.7675
Epoch 196/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4398 - categorical_accuracy: 0.8413 - val_loss: 0.5740 - val_categorical_accuracy: 0.7925
Epoch 197/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4314 - categorical_accuracy: 0.8525 - val_loss: 0.6266 - val_categorical_accuracy: 0.7875
Epoch 198/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4209 - categorical_accuracy: 0.8556 - val_loss: 0.6270 - val_categorical_accuracy: 0.7850
Epoch 199/200
100/100 [==============================] - ETA: 0s - loss: 0.4041 - categorical_accuracy: 0.8531INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Den

INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 51ms/step - loss: 0.4041 - categorical_accuracy: 0.8531 - val_loss: 0.6056 - val_categorical_accuracy: 0.8000
Epoch 200/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4284 - categorical_accuracy: 0.8512 - val_loss: 0.6246 - val_categorical_accuracy: 0.8000


epoch/categorical_accuracy,▁▁▃▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,██▆▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▁▃▄▅▅▅▅▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇██▇█████
epoch/val_loss,██▆▅▄▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▁▁▁▁▁▁
epoch/categorical_accuracy,0.85125
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.42842
epoch/val_categorical_accuracy,0.8


In [29]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM64-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM64-LR-Dense256-fs=36-cls=5",
            USE_WANDB=True)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 64)                433664    
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 64)                0         
                                                                 
 dense_30 (Dense)            (None, 256)               16640     
                                                                 
 dense_31 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 451589 (1.72 MB)
Trainable params: 451589 (1.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/200
     90/Unknown - 2s 5ms/step - loss: 1.6523 - categorical_accuracy: 0.2125

2024-04-24 18:35:54.760681: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1127454815265231155
2024-04-24 18:35:54.760705: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14484767194758726203
2024-04-24 18:35:54.760710: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3296895248915142635
2024-04-24 18:35:54.760719: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11847579756920092354
2024-04-24 18:35:55.278406: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11256574607164290797
2024-04-24 18:35:55.278432: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10123891644433725201
2024-04-24 18:35:55.278437: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=36-cls=5.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=36-cls=5.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 4s 43ms/step - loss: 0.3497 - categorical_accuracy: 0.8881 - val_loss: 0.5954 - val_categorical_accuracy: 0.8150
Epoch 188/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3441 - categorical_accuracy: 0.8900 - val_loss: 0.7073 - val_categorical_accuracy: 0.7750
Epoch 189/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3322 - categorical_accuracy: 0.8919 - val_loss: 0.7158 - val_categorical_accuracy: 0.7650
Epoch 190/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3283 - categorical_accuracy: 0.8944 - val_loss: 0.7366 - val_categorical_accuracy: 0.7550
Epoch 191/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3316 - categorical_accuracy: 0.8950 - val_loss: 0.6856 - val_categorical_accuracy: 0.7875
Epoch 192/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3246 - categorical_accuracy: 0.8963 - val_loss: 0.7815 - val_categorical_accuracy: 0.7350

INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 6s 59ms/step - loss: 0.3172 - categorical_accuracy: 0.9025 - val_loss: 0.6238 - val_categorical_accuracy: 0.8200


epoch/categorical_accuracy,▁▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▂▄▄▄▅▆▅▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█
epoch/val_loss,█▆▅▄▄▃▃▄▂▂▂▁▂▁▁▁▁▁▁▁▂▁▂▁▂▂▁▂▂▂▁▁▁▁▂▂▁▂▂▁
epoch/categorical_accuracy,0.9025
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.31724
epoch/val_categorical_accuracy,0.82


In [30]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=36-cls=5.tf"),
            run_name="AK-LSTM128-LR-Dense256-fs=36-cls=5",
            USE_WANDB=True)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 128)               900096    
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_32 (Dense)            (None, 256)               33024     
                                                                 
 dense_33 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 934405 (3.56 MB)
Trainable params: 934405 (3.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/200
     91/Unknown - 2s 5ms/step - loss: 1.6849 - categorical_accuracy: 0.2246

2024-04-24 18:40:28.555247: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1729799205087681351
2024-04-24 18:40:28.555269: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11256574607164290797
2024-04-24 18:40:28.555276: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10123891644433725201
2024-04-24 18:40:28.555283: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 566830789400684550


INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=36-cls=5.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=36-cls=5.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 4s 44ms/step - loss: 0.4009 - categorical_accuracy: 0.8769 - val_loss: 0.6646 - val_categorical_accuracy: 0.7925
Epoch 96/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3973 - categorical_accuracy: 0.8775 - val_loss: 0.6831 - val_categorical_accuracy: 0.7850
Epoch 97/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3802 - categorical_accuracy: 0.8813 - val_loss: 0.6333 - val_categorical_accuracy: 0.7725
Epoch 98/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3822 - categorical_accuracy: 0.8806 - val_loss: 0.6935 - val_categorical_accuracy: 0.7825
Epoch 99/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3903 - categorical_accuracy: 0.8788 - val_loss: 0.7155 - val_categorical_accuracy: 0.7675
Epoch 100/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3775 - categorical_accuracy: 0.8806 - val_loss: 0.6544 - val_categorical_accuracy: 0.7800
Epo

INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=36-cls=5.tf/assets


100/100 [==============================] - 6s 59ms/step - loss: 0.5234 - categorical_accuracy: 0.8194 - val_loss: 0.6294 - val_categorical_accuracy: 0.8125
Epoch 116/200
100/100 [==============================] - 1s 6ms/step - loss: 0.5414 - categorical_accuracy: 0.8138 - val_loss: 0.6506 - val_categorical_accuracy: 0.7575
Epoch 117/200
100/100 [==============================] - 1s 10ms/step - loss: 0.4933 - categorical_accuracy: 0.8263 - val_loss: 0.6569 - val_categorical_accuracy: 0.7525
Epoch 118/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4952 - categorical_accuracy: 0.8369 - val_loss: 0.6122 - val_categorical_accuracy: 0.7950
Epoch 119/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4752 - categorical_accuracy: 0.8369 - val_loss: 0.7171 - val_categorical_accuracy: 0.7650
Epoch 120/200
100/100 [==============================] - 1s 6ms/step - loss: 0.4866 - categorical_accuracy: 0.8431 - val_loss: 0.6164 - val_categorical_accuracy: 0.787

epoch/categorical_accuracy,▁▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▆▆▇▇▇▇▇▇▇▇▇▇████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▄▄▅▆▆▆▇▇▇▇█▇▇▇▇▇██▇▇████▇▇▇▇▇▇▇███▇▇▇▇
epoch/val_loss,█▅▄▄▃▂▃▂▂▂▁▁▁▁▁▂▂▂▁▂▂▂▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃
epoch/categorical_accuracy,0.94187
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.22237
epoch/val_categorical_accuracy,0.765


In [31]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM256-LR-Dense128-gelu-fs=36-cls=5.tf"),
            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
            USE_WANDB=True)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 36, 256)           1931264   
                                                                 
 flatten_3 (Flatten)         (None, 9216)              0         
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 9216)              0         
                                                                 
 dense_34 (Dense)            (None, 256)               2359552   
                                                                 
 dense_35 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 4292101 (16.37 MB)
Trainable params: 4292101 (16.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/200
     90/Unknown - 2s 5ms/step - loss: 1.6594 - categorical_accuracy: 0.3187

2024-04-24 18:44:22.438185: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14484767194758726203
2024-04-24 18:44:22.438207: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3296895248915142635
2024-04-24 18:44:22.438214: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1618916935498642930
2024-04-24 18:44:22.438224: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11847579756920092354


INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=36-cls=5.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=36-cls=5.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=36-cls=5.tf/assets


100/100 [==============================] - 4s 44ms/step - loss: 0.2680 - categorical_accuracy: 0.9369 - val_loss: 0.9135 - val_categorical_accuracy: 0.8050
Epoch 106/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2477 - categorical_accuracy: 0.9456 - val_loss: 0.9736 - val_categorical_accuracy: 0.7625
Epoch 107/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2483 - categorical_accuracy: 0.9469 - val_loss: 1.0450 - val_categorical_accuracy: 0.7675
Epoch 108/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2297 - categorical_accuracy: 0.9500 - val_loss: 0.9679 - val_categorical_accuracy: 0.7725
Epoch 109/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2254 - categorical_accuracy: 0.9544 - val_loss: 0.9842 - val_categorical_accuracy: 0.7800
Epoch 110/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2561 - categorical_accuracy: 0.9381 - val_loss: 1.0198 - val_categorical_accuracy: 0.7650

INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=36-cls=5.tf/assets


100/100 [==============================] - 5s 53ms/step - loss: 0.0785 - categorical_accuracy: 0.9950 - val_loss: 0.8660 - val_categorical_accuracy: 0.8075
Epoch 190/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0755 - categorical_accuracy: 0.9950 - val_loss: 0.9093 - val_categorical_accuracy: 0.7825
Epoch 191/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0674 - categorical_accuracy: 0.9975 - val_loss: 0.9221 - val_categorical_accuracy: 0.7825
Epoch 192/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0809 - categorical_accuracy: 0.9944 - val_loss: 0.9044 - val_categorical_accuracy: 0.7750
Epoch 193/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0747 - categorical_accuracy: 0.9962 - val_loss: 1.0098 - val_categorical_accuracy: 0.7950
Epoch 194/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0679 - categorical_accuracy: 0.9981 - val_loss: 0.9590 - val_categorical_accuracy: 0.7950

epoch/categorical_accuracy,▁▃▄▅▅▅▆▆▆▇▇▇▇███████▇▇██████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▅▆▆▇▆▇▇▇▇▇▇▇█▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███
epoch/val_loss,█▅▃▄▂▂▁▃▁▂▃▄▄▃▃▂▃▂▃▃▄▃▄▆▄▃▅▃▃▄▄▅▅▅▃▅▄▃▃▄
epoch/categorical_accuracy,0.9975
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.06509
epoch/val_categorical_accuracy,0.7875


In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(512, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM512-LR-Dense128-gelu-fs=36-cls=5.tf"),
            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
            USE_WANDB=True)

## GRU

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU16-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU16-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU32-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU32-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU64-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU64-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU128-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU128-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [ ]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(256, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU256-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU256-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

In [15]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(512, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf"),
            run_name="AK-GRU512-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

Epoch 1/200
     99/Unknown - 2s 5ms/step - loss: 1.7640 - categorical_accuracy: 0.2191

2024-04-24 21:13:10.633795: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8631559620179198724
2024-04-24 21:13:10.633821: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3287054802704920907
2024-04-24 21:13:10.633834: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13924289717184711313
2024-04-24 21:13:10.982364: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6234687711428296893
2024-04-24 21:13:10.982386: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7237066097637090155
2024-04-24 21:13:10.982391: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18166637911478832036
2024-04-24 21:13:10.982399: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv i

INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf/assets


100/100 [==============================] - 4s 23ms/step - loss: 1.7636 - categorical_accuracy: 0.2175 - val_loss: 1.7402 - val_categorical_accuracy: 0.2200
Epoch 2/200
 99/100 [============================>.] - ETA: 0s - loss: 1.7113 - categorical_accuracy: 0.2468INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf/assets


100/100 [==============================] - 2s 24ms/step - loss: 0.1817 - categorical_accuracy: 0.9606 - val_loss: 0.6313 - val_categorical_accuracy: 0.8575
Epoch 166/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2128 - categorical_accuracy: 0.9563 - val_loss: 0.6323 - val_categorical_accuracy: 0.8450
Epoch 167/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2123 - categorical_accuracy: 0.9525 - val_loss: 0.7155 - val_categorical_accuracy: 0.8325
Epoch 168/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1712 - categorical_accuracy: 0.9712 - val_loss: 0.7053 - val_categorical_accuracy: 0.8375
Epoch 169/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1596 - categorical_accuracy: 0.9706 - val_loss: 0.7053 - val_categorical_accuracy: 0.8525
Epoch 170/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1538 - categorical_accuracy: 0.9775 - val_loss: 0.6769 - val_categorical_accuracy: 0.8475

INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-fs=36-cls=5.tf/assets


100/100 [==============================] - 2s 20ms/step - loss: 0.1537 - categorical_accuracy: 0.9744 - val_loss: 0.6147 - val_categorical_accuracy: 0.8600
Epoch 179/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1489 - categorical_accuracy: 0.9750 - val_loss: 0.7224 - val_categorical_accuracy: 0.8225
Epoch 180/200
100/100 [==============================] - 1s 7ms/step - loss: 0.1608 - categorical_accuracy: 0.9725 - val_loss: 0.6922 - val_categorical_accuracy: 0.8375
Epoch 181/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1386 - categorical_accuracy: 0.9794 - val_loss: 0.7616 - val_categorical_accuracy: 0.8275
Epoch 182/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1507 - categorical_accuracy: 0.9750 - val_loss: 0.6868 - val_categorical_accuracy: 0.8375
Epoch 183/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1343 - categorical_accuracy: 0.9819 - val_loss: 0.8142 - val_categorical_accuracy: 0.8225

epoch/categorical_accuracy,▁▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▄▅▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇██▇█████████
epoch/val_loss,█▅▄▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▂▂▁▃▃▂▂▁▃▂▂▂▂▁▂▂▂▂
epoch/categorical_accuracy,0.98125
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.13023
epoch/val_categorical_accuracy,0.83


## Bidirectional LSTM

In [21]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM16-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BLSTM16-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 8ms/step - loss: 1.0840 - categorical_accuracy: 0.4105

2024-04-22 12:38:18.914113: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 12:38:18.914192: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18252922663707797810
2024-04-22 12:38:18.914211: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 12:38:18.914267: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f2044da4f50>, 139805954698224), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f2044da4f50>, 139805954698224), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1ff447c550>, 139775227457840), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1ff591fd90>, 139776573174080), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM16-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM16-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 22s 415ms/step - loss: 0.1709 - categorical_accuracy: 0.9482 - val_loss: 0.2742 - val_categorical_accuracy: 0.9267
Epoch 195/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1977 - categorical_accuracy: 0.9435 - val_loss: 0.2974 - val_categorical_accuracy: 0.9200
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1757 - categorical_accuracy: 0.9518 - val_loss: 0.3225 - val_categorical_accuracy: 0.8867
Epoch 197/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2120 - categorical_accuracy: 0.9365 - val_loss: 0.3382 - val_categorical_accuracy: 0.8800
Epoch 198/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1648 - categorical_accuracy: 0.9553 - val_loss: 0.3465 - val_categorical_accuracy: 0.9000
Epoch 199/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1867 - categorical_accuracy: 0.9553 - val_loss: 0.5068 - val_categorical_accuracy: 0.8733
Epoch

epoch/categorical_accuracy,▁▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█████▇█████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇███
epoch/val_loss,█▇▆▃▄▄▄▃▄▂▂▂▃▃▃▂▂▂▃▃▃▃▃▃▂▁▂▂▃▂▂▃▂▂▂▂▂▂▁▁
epoch/categorical_accuracy,0.94824
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18871
epoch/val_categorical_accuracy,0.91333


In [22]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM32-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BLSTM32-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 9ms/step - loss: 1.0742 - categorical_accuracy: 0.4216

2024-04-22 12:47:50.536797: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14635252564011947125
2024-04-22 12:47:50.536844: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 12:47:50.536871: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe733650>, 139775238372832), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe733650>, 139775238372832), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe7cba10>, 139774325937456), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbdb11d50>, 139774341087840), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM32-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM32-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 21s 393ms/step - loss: 0.2835 - categorical_accuracy: 0.9129 - val_loss: 0.3580 - val_categorical_accuracy: 0.9200
Epoch 76/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2899 - categorical_accuracy: 0.9153 - val_loss: 0.4352 - val_categorical_accuracy: 0.8533
Epoch 77/200
54/54 [==============================] - 1s 11ms/step - loss: 0.3016 - categorical_accuracy: 0.9224 - val_loss: 0.4281 - val_categorical_accuracy: 0.8667
Epoch 78/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2656 - categorical_accuracy: 0.9282 - val_loss: 0.4697 - val_categorical_accuracy: 0.8600
Epoch 79/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2553 - categorical_accuracy: 0.9294 - val_loss: 0.4806 - val_categorical_accuracy: 0.8800
Epoch 80/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2490 - categorical_accuracy: 0.9294 - val_loss: 0.4406 - val_categorical_accuracy: 0.8667
Epoch 81/

epoch/categorical_accuracy,▁▅▆▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▄▄▆▅▄▆▆▆▆▇▇▇▆▇▆█▇█▇▇█▆▇▇█▆▆▇▇▇▇██▇▇███
epoch/val_loss,█▅▆▅▄▅▆▂▄▃▄▂▃▂▃▃▂▁▂▂▃▂▂▄▂▃▃▄▄▂▃▂▂▂▁▃▂▁▁▁
epoch/categorical_accuracy,0.95412
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16424
epoch/val_categorical_accuracy,0.9


In [23]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM64-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-BLSTM64-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 9ms/step - loss: 1.1174 - categorical_accuracy: 0.4645

2024-04-22 12:56:01.007349: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 12:56:01.007443: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 12:56:02.021207: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 12:56:02.021303: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9122064850495178467
2024-04-22 12:56:02.021348: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df5f310>, 139776572418720), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df5f310>, 139776572418720), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9d6a9550>, 139773748536496), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbc7f3ad0>, 139773393444400), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM64-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM64-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 23s 429ms/step - loss: 0.2175 - categorical_accuracy: 0.9471 - val_loss: 0.3553 - val_categorical_accuracy: 0.9267
Epoch 132/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2547 - categorical_accuracy: 0.9329 - val_loss: 0.4296 - val_categorical_accuracy: 0.8733
Epoch 133/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2217 - categorical_accuracy: 0.9494 - val_loss: 0.6109 - val_categorical_accuracy: 0.8667
Epoch 134/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2728 - categorical_accuracy: 0.9259 - val_loss: 0.4151 - val_categorical_accuracy: 0.8667
Epoch 135/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2595 - categorical_accuracy: 0.9282 - val_loss: 0.4296 - val_categorical_accuracy: 0.8933
Epoch 136/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2485 - categorical_accuracy: 0.9424 - val_loss: 0.3412 - val_categorical_accuracy: 0.9067
Epoch

epoch/categorical_accuracy,▁▄▅▆▆▆▇▆▇▇▇▇▇▇▇█████▇▇▇▇█▇▇▇█▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▆▆▄▄▅▅▅▇▇▇▇▇▇█▇▇▇▇▇█▆▆█▇▇▇▆▇██▇▇▇██▇██
epoch/val_loss,█▅▄▄█▇▄▆▅▂▃▂▂▃▂▁▂▂▂▃▂▃▄▂▁▂▂▁▃▂▁▁▂▁▁▁▂▁▂▂
epoch/categorical_accuracy,0.95765
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18169
epoch/val_categorical_accuracy,0.89333


In [24]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM128-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BLSTM128-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     50/Unknown - 5s 9ms/step - loss: 1.1848 - categorical_accuracy: 0.4575

2024-04-22 13:03:09.148544: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:03:09.148639: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 13:03:09.148669: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7134357954231986336
2024-04-22 13:03:09.148747: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 13:03:10.147553: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1570638488140254319
2024-04-22 13:03:10.147610: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 13:03:10.147643: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv it

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9e53ab90>, 139773772335120), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9e53ab90>, 139773772335120), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f6018c8d0>, 139775234956720), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df37c50>, 139774320689760), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM128-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM128-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 23s 433ms/step - loss: 0.1622 - categorical_accuracy: 0.9753 - val_loss: 0.3317 - val_categorical_accuracy: 0.9267
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1656 - categorical_accuracy: 0.9706 - val_loss: 0.3861 - val_categorical_accuracy: 0.9000
Epoch 197/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1566 - categorical_accuracy: 0.9741 - val_loss: 0.4553 - val_categorical_accuracy: 0.8733
Epoch 198/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1637 - categorical_accuracy: 0.9671 - val_loss: 0.4347 - val_categorical_accuracy: 0.9067
Epoch 199/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1584 - categorical_accuracy: 0.9682 - val_loss: 0.3544 - val_categorical_accuracy: 0.9000
Epoch 200/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1665 - categorical_accuracy: 0.9671 - val_loss: 0.4274 - val_categorical_accuracy: 0.8933


epoch/categorical_accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇█████▇▇▇▇▇▇▇▇▇██▇████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▅▇▆▇█▇▇▄▆▇██▇█▇▇▇▇▆▇▆▇▇▆▇▇▇▇█▇▇▇▇█████
epoch/val_loss,█▅▅▃▄▃▂▂▃▆▄▃▂▁▂▂▃▂▂▃▆▂▃▄▂▄▂▃▁▂▁▂▂▃▂▁▁▂▂▂
epoch/categorical_accuracy,0.96706
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16646
epoch/val_categorical_accuracy,0.89333


In [25]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM256-LR-Dense256-gelu-cls="),
            run_name="AK-BLSTM256-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     49/Unknown - 5s 9ms/step - loss: 1.2938 - categorical_accuracy: 0.4554

2024-04-22 13:10:18.669765: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c7593d0>, 139774286764992), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c7593d0>, 139774286764992), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c78a490>, 139772389045552), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f859cc750>, 139772734880816), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM256-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM256-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 22s 406ms/step - loss: 0.2760 - categorical_accuracy: 0.9447 - val_loss: 0.4081 - val_categorical_accuracy: 0.9200
Epoch 88/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2635 - categorical_accuracy: 0.9565 - val_loss: 0.5574 - val_categorical_accuracy: 0.8600
Epoch 89/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2437 - categorical_accuracy: 0.9647 - val_loss: 0.4293 - val_categorical_accuracy: 0.9133
Epoch 90/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2807 - categorical_accuracy: 0.9459 - val_loss: 0.5564 - val_categorical_accuracy: 0.8800
Epoch 91/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2847 - categorical_accuracy: 0.9471 - val_loss: 0.4785 - val_categorical_accuracy: 0.8667
Epoch 92/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2562 - categorical_accuracy: 0.9600 - val_loss: 0.4919 - val_categorical_accuracy: 0.8867
Epoch 93/

epoch/categorical_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇██▇█████▇▇▇▇▇█▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▃▄▄▇▇▅▇▆▃▇▆▆███▇▇▇▇▄▇▄▆▆▇█▇▅▅▆▇▆▇▇▇█▇█
epoch/val_loss,█▅▇▆▅▃▄▅▂▄▇▃▃▃▂▂▃▃▃▂▁▇▂▆▃▃▃▁▂▆▅▃▂▃▂▃▂▂▁▂
epoch/categorical_accuracy,0.97529
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.15214
epoch/val_categorical_accuracy,0.89333


## Bidirectional GRU

In [14]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

keras_train(model, filepath=os.path.join("models", "AK-BGRU16-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU16-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

wandb: Currently logged in as: mlewand (mlewand7). Use `wandb login --relogin` to force relogin


Epoch 1/200


2024-04-24 21:08:43.586402: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-24 21:08:45.741649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2024-04-24 21:08:46.286642: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff91cb385d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-24 21:08:46.286656: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-24 21:08:46.289700: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-24 21:08:46.351098: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


     97/Unknown - 4s 5ms/step - loss: 1.6177 - categorical_accuracy: 0.2577

2024-04-24 21:08:47.081073: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 680343906360256750
2024-04-24 21:08:47.081104: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5568635252798777670
2024-04-24 21:08:47.081116: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14554434588429036870
2024-04-24 21:08:47.604985: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15160135340879140909
2024-04-24 21:08:47.605008: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10937198591851436308
2024-04-24 21:08:47.605012: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13129864008515888114
2024-04-24 21:08:47.605019: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff9ebf92310>, 140712585000672), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU16-LR-Dense256-gelu-fs=36-cls=5/assets


INFO:tensorflow:Assets written to: models/AK-BGRU16-LR-Dense256-gelu-fs=36-cls=5/assets


100/100 [==============================] - 5s 51ms/step - loss: 0.1622 - categorical_accuracy: 0.9750 - val_loss: 0.5900 - val_categorical_accuracy: 0.8425
Epoch 183/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1550 - categorical_accuracy: 0.9731 - val_loss: 0.6724 - val_categorical_accuracy: 0.8150
Epoch 184/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1456 - categorical_accuracy: 0.9700 - val_loss: 0.6108 - val_categorical_accuracy: 0.8100
Epoch 185/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1578 - categorical_accuracy: 0.9737 - val_loss: 0.6518 - val_categorical_accuracy: 0.8225
Epoch 186/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1539 - categorical_accuracy: 0.9688 - val_loss: 0.6300 - val_categorical_accuracy: 0.8225
Epoch 187/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1595 - categorical_accuracy: 0.9706 - val_loss: 0.6749 - val_categorical_accuracy: 0.8025

epoch/categorical_accuracy,▁▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█▇█▇▇█▇▇█▇▇█▇███▇████
epoch/val_loss,█▅▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▁▂▂▁▂▁▂▁▂▁▂▂▂
epoch/categorical_accuracy,0.97188
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.13875
epoch/val_categorical_accuracy,0.8275


In [35]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU32-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU32-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

Epoch 1/200
     90/Unknown - 4s 5ms/step - loss: 1.6104 - categorical_accuracy: 0.2910

2024-04-24 19:08:53.992789: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11091856310625635489


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 96), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bdee71bd0>, 140101272257904), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 96), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bdee71bd0>, 140101272257904), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32, 96), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bd2b7d610>, 140101272984736), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6c0ddb9650>, 140102056108208), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU32-LR-Dense256-gelu-fs=36-cls=5/assets


INFO:tensorflow:Assets written to: models/AK-BGRU32-LR-Dense256-gelu-fs=36-cls=5/assets


100/100 [==============================] - 13s 129ms/step - loss: 0.2832 - categorical_accuracy: 0.9200 - val_loss: 0.6033 - val_categorical_accuracy: 0.8075
Epoch 66/200
100/100 [==============================] - 1s 6ms/step - loss: 0.3116 - categorical_accuracy: 0.9206 - val_loss: 0.6077 - val_categorical_accuracy: 0.7850
Epoch 67/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2841 - categorical_accuracy: 0.9275 - val_loss: 0.6710 - val_categorical_accuracy: 0.7800
Epoch 68/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2722 - categorical_accuracy: 0.9306 - val_loss: 0.6017 - val_categorical_accuracy: 0.8075
Epoch 69/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2757 - categorical_accuracy: 0.9312 - val_loss: 0.6516 - val_categorical_accuracy: 0.7800
Epoch 70/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2614 - categorical_accuracy: 0.9325 - val_loss: 0.6272 - val_categorical_accuracy: 0.7850
Ep

epoch/categorical_accuracy,▁▃▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▅▅▆▇▇▇▇▇▇▇▇█▇▇▇█▇▇█▇▇▇█▇▇▇███▇█▇█▇▇▇▇█
epoch/val_loss,█▅▄▃▃▂▂▂▂▂▃▃▂▁▂▂▂▂▂▂▃▄▂▃▂▂▃▃▃▃▃▃▃▄▃▄▃▂▃▃
epoch/categorical_accuracy,0.98188
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.11626
epoch/val_categorical_accuracy,0.79


In [36]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU64-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU64-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

Epoch 1/200
    100/Unknown - 5s 5ms/step - loss: 1.6366 - categorical_accuracy: 0.3000

2024-04-24 19:14:53.393578: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4517613894368094407
2024-04-24 19:14:53.393599: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11748927709039466247
2024-04-24 19:14:53.393605: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15081470965041692872
2024-04-24 19:14:53.393608: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16849516376404360284
2024-04-24 19:14:54.076394: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1651810212778219037
2024-04-24 19:14:54.076420: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5072822412923122032
2024-04-24 19:14:54.076431: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 192), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bdc408650>, 140102038075328), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 192), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bdc408650>, 140102038075328), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 192), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6c1520db10>, 140102039686704), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bd2ff4750>, 140101232126000), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU64-LR-Dense256-gelu-fs=36-cls=5/assets


INFO:tensorflow:Assets written to: models/AK-BGRU64-LR-Dense256-gelu-fs=36-cls=5/assets


100/100 [==============================] - 13s 131ms/step - loss: 0.2260 - categorical_accuracy: 0.9594 - val_loss: 0.7132 - val_categorical_accuracy: 0.8225
Epoch 110/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1878 - categorical_accuracy: 0.9619 - val_loss: 0.8990 - val_categorical_accuracy: 0.7800
Epoch 111/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1760 - categorical_accuracy: 0.9706 - val_loss: 0.9128 - val_categorical_accuracy: 0.7600
Epoch 112/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1731 - categorical_accuracy: 0.9638 - val_loss: 0.8418 - val_categorical_accuracy: 0.7725
Epoch 113/200
100/100 [==============================] - 1s 6ms/step - loss: 0.2027 - categorical_accuracy: 0.9600 - val_loss: 0.8165 - val_categorical_accuracy: 0.7425
Epoch 114/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1944 - categorical_accuracy: 0.9650 - val_loss: 0.8609 - val_categorical_accuracy: 0.75

epoch/categorical_accuracy,▁▃▅▅▆▆▆▇▇▇▇▇▇███████▇▇██████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▅▅▆▆▇▆▆▇▆▇▇█▇▇▆▇▇▇██▇▇▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇█
epoch/val_loss,█▄▃▂▂▂▂▂▃▂▂▂▂▂▂▃▅▃▃▃▁▂▃▃▃▃▆▅▄▃▅▄▆▆▆▆▅▄▃▃
epoch/categorical_accuracy,0.995
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.08504
epoch/val_categorical_accuracy,0.7775


In [37]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU128-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU128-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

Epoch 1/200
     93/Unknown - 4s 5ms/step - loss: 1.6822 - categorical_accuracy: 0.3253INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bcdffb790>, 140103151503264), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bcdffb790>, 140103151503264), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bc3836950>, 140101268084400), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bc3836950>, 140101268084400), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(2, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6c0ee25f10>, 140103151514912), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bd37eb910>, 140102059241232), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU128-LR-Dense256-gelu-fs=36-cls=5/assets


INFO:tensorflow:Assets written to: models/AK-BGRU128-LR-Dense256-gelu-fs=36-cls=5/assets


100/100 [==============================] - 12s 122ms/step - loss: 0.0715 - categorical_accuracy: 0.9994 - val_loss: 0.8877 - val_categorical_accuracy: 0.7950
Epoch 187/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0823 - categorical_accuracy: 0.9962 - val_loss: 0.9746 - val_categorical_accuracy: 0.7675
Epoch 188/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0720 - categorical_accuracy: 0.9981 - val_loss: 0.9445 - val_categorical_accuracy: 0.7700
Epoch 189/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0710 - categorical_accuracy: 0.9987 - val_loss: 0.9507 - val_categorical_accuracy: 0.7825
Epoch 190/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0741 - categorical_accuracy: 0.9987 - val_loss: 0.9035 - val_categorical_accuracy: 0.7825
Epoch 191/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0744 - categorical_accuracy: 0.9956 - val_loss: 0.8910 - val_categorical_accuracy: 0.77

epoch/categorical_accuracy,▁▄▅▅▆▆▆▇▇▇▇█████████████████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▆▆▆▆▆▇▇▇▆▇▇▆▆▆▆▇▇▇▇▇▆█▇▇▇▇▇▇▇▇▇█▇█▇██
epoch/val_loss,█▅▂▃▃▃▄▃▃▁▂▃▃▄▄▃▃▃▂▃▃▂▂▄▂▄▃▄▃▄▄▅▅▄▄▃▃▃▂▂
epoch/categorical_accuracy,0.99875
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.06862
epoch/val_categorical_accuracy,0.795


In [38]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU256-LR-Dense256-gelu-fs=36-cls=5"),
            run_name="AK-BGRU256-LR-Dense256-gelu-fs=36-cls=5",
            USE_WANDB=True)

Epoch 1/200
     98/Unknown - 4s 6ms/step - loss: 1.7290 - categorical_accuracy: 0.3438

2024-04-24 19:26:30.750764: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6848965035515004211
2024-04-24 19:26:30.750794: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8467865002374872757
2024-04-24 19:26:30.750803: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1651810212778219037


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 768), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6c0f366490>, 140101017959792), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 768), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6c0f366490>, 140101017959792), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 768), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bd0abbe10>, 140100928372400), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6bdf3d5950>, 140100562186000), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU256-LR-Dense256-gelu-fs=36-cls=5/assets


INFO:tensorflow:Assets written to: models/AK-BGRU256-LR-Dense256-gelu-fs=36-cls=5/assets


100/100 [==============================] - 14s 137ms/step - loss: 0.0788 - categorical_accuracy: 0.9987 - val_loss: 0.8354 - val_categorical_accuracy: 0.7925
Epoch 188/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0758 - categorical_accuracy: 0.9994 - val_loss: 0.9317 - val_categorical_accuracy: 0.7675
Epoch 189/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0739 - categorical_accuracy: 0.9994 - val_loss: 0.9450 - val_categorical_accuracy: 0.7775
Epoch 190/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0748 - categorical_accuracy: 0.9994 - val_loss: 0.9150 - val_categorical_accuracy: 0.7750
Epoch 191/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0767 - categorical_accuracy: 0.9981 - val_loss: 0.9339 - val_categorical_accuracy: 0.7600
Epoch 192/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0728 - categorical_accuracy: 1.0000 - val_loss: 0.9644 - val_categorical_accuracy: 0.77

epoch/categorical_accuracy,▁▄▅▆▆▇▇▇████████████▇███████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▅▅▆▅▅▆▆▇▆▆▆▆▆▆▇▆▇▇▆▇█▇▇▇▆▆███▇██▇▇███
epoch/val_loss,█▇▂▃▅▃▆▆▄▃▂▅▂▄▃▄▅▃▄▂▁▇▄▃▅▅▃▄▄▂▃▃▃▃▃▄▄▂▂▁
epoch/categorical_accuracy,0.99937
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.07249
epoch/val_categorical_accuracy,0.775


In [17]:
X_shape = train_dataset_parquet[0][0].shape
Y_shape = train_dataset_parquet[0][1].shape
print(f"X_shape = {X_shape}, Y_shape = {Y_shape}")

X_shape = (36, 1629), Y_shape = (5,)


In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


In [19]:
embed_dim = 64  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside 

inputs = layers.Input(shape=X_shape)
embedding_layer = TokenAndPositionEmbedding(X_shape[0], X_shape[1])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(X_shape[1], num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = Dense(len(train_dataset_parquet.unique_labels), activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

keras_train(model, filepath=os.path.join("models", f"Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128-LipsEyesHandsPose_5k.tf"),
            run_name=f"Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128-LipsEyesHandsPose_5k",
                max_lr=1e-4,
            min_lr=2.5e-5,
            T_max=101,
            epochs=200,
            mediapipe_features="reduced-LipsEyesHandsPose-scaled",
            USE_WANDB=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 36, 1629)          21665828  
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 1629)              0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 1629)              0     

Epoch 1/200
     96/Unknown - 4s 11ms/step - loss: 1.6830 - categorical_accuracy: 0.2227

2024-04-24 21:16:30.370882: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17501152575416454053
2024-04-24 21:16:30.370907: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6029543321916488959
2024-04-24 21:16:30.370912: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 145402771508560368
2024-04-24 21:16:30.370921: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10502063042031140596
2024-04-24 21:16:30.674387: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17501152575416454053
2024-04-24 21:16:30.674412: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8074986277270048333
2024-04-24 21:16:30.674420: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv i

INFO:tensorflow:Assets written to: models/Transformer-e=64-h=2-ff=128-D=0.1-Dense=128-LipsEyesHandsPose_5k.tf/assets


INFO:tensorflow:Assets written to: models/Transformer-e=64-h=2-ff=128-D=0.1-Dense=128-LipsEyesHandsPose_5k.tf/assets


...

INFO:tensorflow:Assets written to: models/Transformer-e=64-h=2-ff=128-D=0.1-Dense=128-LipsEyesHandsPose_5k.tf/assets


100/100 [==============================] - 2s 24ms/step - loss: 0.1966 - categorical_accuracy: 0.9281 - val_loss: 0.5096 - val_categorical_accuracy: 0.8475
Epoch 96/200
100/100 [==============================] - 1s 13ms/step - loss: 0.1773 - categorical_accuracy: 0.9413 - val_loss: 0.6925 - val_categorical_accuracy: 0.8075
Epoch 97/200
100/100 [==============================] - 1s 13ms/step - loss: 0.1878 - categorical_accuracy: 0.9350 - val_loss: 0.5710 - val_categorical_accuracy: 0.8275
Epoch 98/200
100/100 [==============================] - 1s 13ms/step - loss: 0.1968 - categorical_accuracy: 0.9362 - val_loss: 0.7599 - val_categorical_accuracy: 0.7950
Epoch 99/200
100/100 [==============================] - 1s 13ms/step - loss: 0.1703 - categorical_accuracy: 0.9400 - val_loss: 0.6631 - val_categorical_accuracy: 0.7950
Epoch 100/200
100/100 [==============================] - 1s 13ms/step - loss: 0.1840 - categorical_accuracy: 0.9394 - val_loss: 0.6428 - val_categorical_accuracy: 0.832

INFO:tensorflow:Assets written to: models/Transformer-e=64-h=2-ff=128-D=0.1-Dense=128-LipsEyesHandsPose_5k.tf/assets


100/100 [==============================] - 3s 26ms/step - loss: 0.1117 - categorical_accuracy: 0.9581 - val_loss: 0.5932 - val_categorical_accuracy: 0.8700
Epoch 176/200
100/100 [==============================] - 1s 13ms/step - loss: 0.1197 - categorical_accuracy: 0.9669 - val_loss: 0.6912 - val_categorical_accuracy: 0.8375
Epoch 177/200
100/100 [==============================] - 1s 13ms/step - loss: 0.0910 - categorical_accuracy: 0.9669 - val_loss: 0.7345 - val_categorical_accuracy: 0.8275
Epoch 178/200
100/100 [==============================] - 1s 13ms/step - loss: 0.0913 - categorical_accuracy: 0.9744 - val_loss: 0.5703 - val_categorical_accuracy: 0.8600
Epoch 179/200
100/100 [==============================] - 1s 13ms/step - loss: 0.0858 - categorical_accuracy: 0.9700 - val_loss: 0.6224 - val_categorical_accuracy: 0.8400
Epoch 180/200
100/100 [==============================] - 1s 13ms/step - loss: 0.0898 - categorical_accuracy: 0.9737 - val_loss: 0.6582 - val_categorical_accuracy: 0

epoch/categorical_accuracy,▁▂▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇▇███████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▇▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▆▆▇▇▆▇▇▇████▇▇▇█▇█▇▇██▇██████▇███████
epoch/val_loss,█▆▄▃▃▂▂▃▂▂▂▁▁▁▁▂▃▂▂▃▂▃▂▁▁▂▂▁▁▂▂▃▃▂▂▂▂▂▂▃
epoch/categorical_accuracy,0.97688
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.06767
epoch/val_categorical_accuracy,0.8475


In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 36, 1629)          21665828  
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 1629)              0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 1629)              0     